In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes rouge-score nltk

In [ ]:
from datasets import load_dataset

dataset = load_dataset("iamtarun/code_instructions_120k_alpaca")

print("Dataset loaded ✅")

In [ ]:
def filter_python(example):
    return "def " in example["output"] or "import " in example["output"]

filtered = dataset["train"].filter(filter_python)

print("Filtered size:", len(filtered))

In [ ]:
filtered = filtered.shuffle(seed=42).select(range(4000))

# Split into train / validation / test
split = filtered.train_test_split(test_size=0.2, seed=42)
test_valid = split["test"].train_test_split(test_size=0.5, seed=42)

train_dataset = split["train"]
val_dataset = test_valid["train"]
test_dataset = test_valid["test"]

print("Train:", len(train_dataset))
print("Validation:", len(val_dataset))
print("Test:", len(test_dataset))

In [ ]:
def format_example(example):
    text = f"""Below is an instruction that describes a task.

### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:
{example['output']}"""

    return {"text": text}

train_dataset = train_dataset.map(format_example)
val_dataset = val_dataset.map(format_example)

print("Columns:", train_dataset.column_names)

In [ ]:
# IMPORT TOKENIZER
from transformers import AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

print("Tokenizer loaded successfully ✅")

In [ ]:


def tokenize_function(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        max_length=256,
        padding="max_length"
    )

    # Labels required for training
    tokens["labels"] = tokens["input_ids"].copy()

    return tokens

train_dataset = train_dataset.map(
    tokenize_function,
    remove_columns=train_dataset.column_names
)

val_dataset = val_dataset.map(
    tokenize_function,
    remove_columns=val_dataset.column_names
)

print("Columns:", train_dataset.column_names)

In [ ]:


from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float32
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Set pad token
model.config.pad_token_id = tokenizer.eos_token_id

print("Model loaded with quantization ✅")

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare model for QLoRA training
model = prepare_model_for_kbit_training(model)

# Define QLoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply adapters
model = get_peft_model(model, lora_config)

# Show trainable params
model.print_trainable_parameters()

print("QLoRA adapters applied successfully ✅")

In [ ]:


from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./qlora-output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    max_steps=100,            # fast training for MSc project
    logging_steps=10,
    save_steps=50,
    save_total_limit=1,
    fp16=False,               # safe setting
    report_to="none",
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

print("Training started...")

trainer.train()

print("Training completed successfully ✅")

In [ ]:

prompt = """Below is an instruction.

### Instruction:
Write a Python function to add two numbers.

### Input:


### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

output = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.7,
    top_p=0.9
)

generated_code = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Output:\n")
print(generated_code)

In [ ]:
def add(a, b):
    return a + b

In [ ]:
from evaluate import load

bleu = load("bleu")
rouge = load("rouge")

prediction = generated_code

reference = """def add(a, b):
    return a + b"""

bleu_score = bleu.compute(
    predictions=[prediction],
    references=[[reference]]
)

rouge_score = rouge.compute(
    predictions=[prediction],
    references=[reference]
)

print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)

In [ ]:
model.save_pretrained("./qlora-trained-model")
tokenizer.save_pretrained("./qlora-trained-model")

print("QLoRA model saved successfully ✅")